In [5]:
import numpy as np
import pandas as pd
import os

In [6]:
def unwrap_trajectory(particle_positions, box_sizes):
    # Compute displacement between frames
    delta_x = np.diff(particle_positions, axis=0)
    
    # Get nearest image displacement
    delta_x = np.fix(2 * delta_x / box_sizes) * box_sizes - delta_x
    
    # Take cumulative sum of displacements
    unwrapped_positions = np.cumsum(delta_x, axis=0)
    
    return unwrapped_positions

def compute_mean_displacement(unwrapped_positions):
    # Average trajectories over all particles
    mean_displacement = np.mean(unwrapped_positions, axis=1)
    
    return mean_displacement

def compute_velocity(mean_displacement, time_interval):
    # Fit a straight line to the displacement to compute velocity
    velocity = np.polyfit(np.arange(len(mean_displacement)), mean_displacement, 1)[0]
    
    # Convert displacement to velocity (assuming time_interval is 1)
    velocity /= time_interval
    
    return velocity

In [7]:
if __name__ == "__main__":
    directory = "."  # Directory containing the data files
    time_interval = 1
    
    results = []  # List to store results
    
    for filename in os.listdir(directory):
        if filename.startswith("sed_N8000_phi") and filename.endswith("_HI.txt"):
            print(f"Processing file: {filename}")  # Print filename before loading data
            
            # Extract phi from filename
            phi = float(filename.split("_")[2].replace("phi", ""))
            
            tag = 0

            with open(os.path.join(directory, filename), 'r') as file:
                
                # Initialize arrays to store particle positions for each frame
                particle_positions = []
                box_sizes = None

                # Read the file line by line
                frame_positions = []
                row_count = 0
                for line in file:
                    if row_count % 8003 == 0:
                        N = int(line)
                    elif row_count % 8003 == 1:
                        box_sizes = np.array([float(x) for x in line.split()])
                    elif line.strip():
                        frame_positions.append([float(x) for x in line.split()])
                    else:
                        # Process the completed frame
                        particle_positions.append(np.array(frame_positions))
                        frame_positions = []
                    row_count += 1

                # Convert particle positions to numpy array
                particle_positions = np.array(particle_positions)
            
                # Initialize arrays to store mean displacements and velocities for each frame
                mean_displacements = []
                velocities = []
            
                unwrapped_positions = unwrap_trajectory(particle_positions, box_sizes)
                    
                # Compute mean displacement for the frame
                mean_displacement = compute_mean_displacement(unwrapped_positions)
                
                # Compute velocity for the frame
                velocity = compute_velocity(mean_displacement, time_interval)

                # Save velocity components along with phi
                results.append([phi, velocity[0], velocity[1], velocity[2]])

                print(f"For phi={phi}: Average velocity = {velocity}")
    
    # Sort results based on phi values
    results.sort(key=lambda x: x[0])
    
    # Write results to a text file
    with open("computational_velocity_HI.txt", "w") as outfile:
        outfile.write("Phi\tVelocity_X\tVelocity_Y\tVelocity_Z\n")
        for result in results:
            outfile.write(f"{result[0]}\t{result[1]}\t{result[2]}\t{result[3]}\n")
    
    print("Results saved to computational_velocity.txt")


Processing file: sed_N8000_phi0.04_HI.txt
For phi=0.04: Average velocity = [ 0.00680396 -0.00828404  0.79529327]
Processing file: sed_N8000_phi0.20_HI.txt
For phi=0.2: Average velocity = [0.00077525 0.00167185 0.35992032]
Processing file: sed_N8000_phi0.30_HI.txt
For phi=0.3: Average velocity = [ 0.00532318 -0.00232522  0.22502775]
Processing file: sed_N8000_phi0.45_HI.txt
For phi=0.45: Average velocity = [ 0.00478617 -0.00124939  0.13566611]
Processing file: sed_N8000_phi0.02_HI.txt
For phi=0.02: Average velocity = [ 0.01320008 -0.00841104  0.88447914]
Processing file: sed_N8000_phi0.10_HI.txt
For phi=0.1: Average velocity = [ 0.00517888 -0.00370612  0.59661805]
Processing file: sed_N8000_phi0.50_HI.txt
For phi=0.5: Average velocity = [-0.00015373  0.01038349  0.11970644]
Processing file: sed_N8000_phi0.05_HI.txt
For phi=0.05: Average velocity = [ 0.00585214 -0.00639812  0.75277192]
Processing file: sed_N8000_phi0.40_HI.txt
For phi=0.4: Average velocity = [ 0.00452226 -0.00168015  0.1